https://ychung38.medium.com/how-to-use-solver-excel-in-python-458336408c7f

In [2]:
# import sys
# !{sys.executable} -m pip install PuLP

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame({'Variable': ['x1', 'x2'],
                   'Price': [45, 20],
                   'Cost': [30, 10],
                   'Demand': [2000, 8000]
                    })

df.eval('Margin=Price-Cost', inplace=True)

In [5]:
df

,Variable,Price,Cost,Demand,Margin
0,x1,45,30,2000,15
1,x2,20,10,8000,10


In [6]:
from pulp import *

In [7]:
# set the dictionary for each feature
prob = LpProblem('Sell', LpMaximize) # Objective function

inv_items = list(df['Variable']) # Variable name

margin = dict(zip(inv_items, df['Margin'])) # Function 

demand = dict(zip(inv_items, df['Demand'])) # Function

print(margin, demand)

# next, we are defining our decision variables as investments and are adding a few parameters to it
inv_vars = LpVariable.dicts('Vairable', inv_items, lowBound=0, cat='Integer')

# set the decision variables
# all add in the problem setting
prob += lpSum([inv_vars[i] * margin[i] for i in inv_items])

# Constraint
prob += lpSum([inv_vars[i] for i in inv_items]) <= 8000, 'Total Demand'

# Constraint
prob += inv_vars['x1'] <= 2000, 'x1 Demand'

# Constraint
prob += inv_vars['x2'] <= 8000, 'x2 Demand'

{'x1': 15, 'x2': 10} {'x1': 2000, 'x2': 8000}


In [8]:
prob

Sell:
MAXIMIZE
15*Vairable_x1 + 10*Vairable_x2 + 0
SUBJECT TO
Total_Demand: Vairable_x1 + Vairable_x2 <= 8000

x1_Demand: Vairable_x1 <= 2000

x2_Demand: Vairable_x2 <= 8000

VARIABLES
0 <= Vairable_x1 Integer
0 <= Vairable_x2 Integer

In [9]:
prob.solve()

1

In [10]:

# Answer
value(prob.objective) # 9000
# Variables' values
print('The optimal answer\n'+'-'*70)
for v in prob.variables():
    if v.varValue > 0:
       print(v.name, '=', v.varValue)

The optimal answer
----------------------------------------------------------------------
Vairable_x1 = 2000.0
Vairable_x2 = 6000.0


### Example

In [14]:
LENGTH = 6

Variables = ['x'+str(i+1) for i in range(LENGTH)]

PROFIT = [7, 18, 30, 8, 15, 6]

Vars2Dict = {}
for i in range(LENGTH):
    Vars2Dict[i] = Variables[i]

Demand = [7, 6, 5, 4, 3, 2]

Length = {
    '12': 1,
    '23': 2,
    '34': 3 
}

Capacity = {
    '12': 7,
    '23': 8,
    '34': 9 
}

Legs = {
    '12':[1,1,1,0,0,0],
    '23':[0,1,1,1,1,0],
    '34':[0,0,1,0,1,1]
}

LEGS = list(Legs.keys())

print(Vars2Dict)
Constraints = {}
for LEG in LEGS:
    Constraints[LEG] = []
    for Index, L in enumerate(Legs[LEG]):
        if L:
            Constraints[LEG].append(Vars2Dict[Index]) 
Constraints            

{0: 'x1', 1: 'x2', 2: 'x3', 3: 'x4', 4: 'x5', 5: 'x6'}


{'12': ['x1', 'x2', 'x3'],
 '23': ['x2', 'x3', 'x4', 'x5'],
 '34': ['x3', 'x5', 'x6']}

In [15]:
df = pd.DataFrame({'Variable': Variables,
                   'Demand': Demand
                    })

df

,Variable,Demand
0,x1,7
1,x2,6
2,x3,5
3,x4,4
4,x5,3
5,x6,2


In [18]:
prob = LpProblem('Deterministic_Model', LpMaximize)

variables = list(df['Variable'])
varsDict = LpVariable.dicts('Variable', variables, lowBound=0, cat='Integer')
print(varsDict)

prob += lpSum([varsDict[v] * PROFIT[i] for i, v in enumerate(variables)])

# Capacity Constraint on each Leg
for Index, LEG in enumerate(LEGS):
    prob += lpSum([varsDict[v] * Legs[LEG][i] for i, v in enumerate(variables)]) <= Capacity[LEG], 'Leg '+ LEG 

# Demand Constraint on each OD Trip
for Index, V in enumerate(variables):
    prob += varsDict[V] <= Demand[Index], V
    
prob

{'x1': Variable_x1, 'x2': Variable_x2, 'x3': Variable_x3, 'x4': Variable_x4, 'x5': Variable_x5, 'x6': Variable_x6}


Deterministic_Model:
MAXIMIZE
7*Variable_x1 + 18*Variable_x2 + 30*Variable_x3 + 8*Variable_x4 + 15*Variable_x5 + 6*Variable_x6 + 0
SUBJECT TO
Leg_12: Variable_x1 + Variable_x2 + Variable_x3 <= 7

Leg_23: Variable_x2 + Variable_x3 + Variable_x4 + Variable_x5 <= 8

Leg_34: Variable_x3 + Variable_x5 + Variable_x6 <= 9

x1: Variable_x1 <= 7

x2: Variable_x2 <= 6

x3: Variable_x3 <= 5

x4: Variable_x4 <= 4

x5: Variable_x5 <= 3

x6: Variable_x6 <= 2

VARIABLES
0 <= Variable_x1 Integer
0 <= Variable_x2 Integer
0 <= Variable_x3 Integer
0 <= Variable_x4 Integer
0 <= Variable_x5 Integer
0 <= Variable_x6 Integer

In [19]:
prob.solve()

1

In [20]:
Objective = value(prob.objective)
print('Objective: ', Objective)

Objective:  217.0


In [21]:
for v in prob.variables():
    print(v.name, '=', v.varValue)

Variable_x1 = 1.0
Variable_x2 = 1.0
Variable_x3 = 5.0
Variable_x4 = 0.0
Variable_x5 = 2.0
Variable_x6 = 2.0
